In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

import os
import numpy as np
import matplotlib.pyplot as plt


In [2]:
train_dir='class_data/train'
validation_dir='class_data/validation'
test_dir='class_data/test'

In [3]:
batch_size=100
epochs=50
IMG_HEIGHT=125
IMG_WIDTH=125

In [4]:
train_image_generator=ImageDataGenerator(
                        rescale=1./255,
                        rotation_range=45,
                        width_shift_range=.15,
                        height_shift_range=.15,
                        horizontal_flip=True,
                        zoom_range=0.5
                        )
validation_image_generator=ImageDataGenerator(rescale=1./255)
test_image_generator=ImageDataGenerator(rescale=1./255)

In [5]:
train_data_gen=train_image_generator.flow_from_directory(batch_size=batch_size,directory=train_dir,shuffle=True,target_size=(IMG_HEIGHT,IMG_WIDTH),class_mode='binary',color_mode='grayscale')

Found 15000 images belonging to 2 classes.


In [6]:
validation_data_gen=validation_image_generator.flow_from_directory(batch_size=batch_size,directory=validation_dir,shuffle=True,target_size=(IMG_HEIGHT,IMG_WIDTH),class_mode='binary',color_mode='grayscale')

Found 5000 images belonging to 2 classes.


In [7]:
test_data_gen=test_image_generator.flow_from_directory(batch_size=batch_size,directory=test_dir,shuffle=True,target_size=(IMG_HEIGHT,IMG_WIDTH),class_mode='binary',color_mode='grayscale')

Found 5000 images belonging to 2 classes.


In [8]:
tf.distribute.get_strategy()

In [9]:
model=Sequential([
    Conv2D(16,5,padding='same',activation='relu',input_shape=(IMG_HEIGHT,IMG_WIDTH,1)),
    MaxPooling2D(),
    Conv2D(32,5,padding='same',activation='relu'),
    MaxPooling2D(),
    Conv2D(64,5,padding='same',activation='relu'),
    MaxPooling2D(),
    Conv2D(128,5,padding='same',activation='relu'),
    MaxPooling2D(),
    Dropout(0.2),
    Flatten(),
    Dense(512,activation='relu'),
    Dense(1)
    
])

model.compile(optimizer='adam',
             loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
             metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 125, 125, 16)      416       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 62, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 31, 31, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 15, 15, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       2

In [ ]:
# model=tf.keras.models.load_model('tf_model/MyModel')

history=model.fit(
    train_data_gen,
    steps_per_epoch=int(15000/batch_size),
    epochs=epochs,
    validation_data=validation_data_gen,
    validation_steps=int(5000/batch_size)
)

Train for 150 steps, validate for 50 steps
Epoch 1/50
150/150 [==============================] - 281s 2s/step - loss: 0.6911 - accuracy: 0.5062 - val_loss: 0.7016 - val_accuracy: 0.5936
Epoch 2/50
150/150 [==============================] - 276s 2s/step - loss: 0.6850 - accuracy: 0.5025 - val_loss: 0.6816 - val_accuracy: 0.5000
Epoch 3/50
150/150 [==============================] - 278s 2s/step - loss: 0.6737 - accuracy: 0.5414 - val_loss: 0.6608 - val_accuracy: 0.5564
Epoch 4/50
150/150 [==============================] - 279s 2s/step - loss: 0.6585 - accuracy: 0.5762 - val_loss: 0.6303 - val_accuracy: 0.6178
Epoch 5/50
150/150 [==============================] - 286s 2s/step - loss: 0.6522 - accuracy: 0.5861 - val_loss: 0.6121 - val_accuracy: 0.6598
Epoch 6/50
150/150 [==============================] - 291s 2s/step - loss: 0.6372 - accuracy: 0.6121 - val_loss: 0.6099 - val_accuracy: 0.7012
Epoch 7/50
150/150 [==============================] - 316s 2s/step - loss: 0.6135 - accuracy: 0.637

In [ ]:
model.save('gray_tf_model/MyModel',save_format='tf')

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

In [ ]:
sample_test_image,_=next(test_data_gen)


In [ ]:
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip( images_arr, axes):
        print(np.shape(img[:,:,0]))
        
        ax.imshow(img[:,:,0],cmap='gray',vmin=0,vmax=1)
        ax.axis('off')
    plt.tight_layout()
    plt.show()


plotImages(sample_test_image[:5])

In [ ]:
model.predict(sample_test_image[:1])
print(np.shape(sample_test_image[:1]))

In [ ]:
output=model.evaluate(test_data_gen)

In [ ]:
print(output)

In [ ]:
np.shape(sample_test_image)

In [ ]:
import time
def plot_with_tag(img_arr):
    for img in img_arr:
        pred=model.predict(np.reshape(img,(1,100,100,1)))
        plt.title('dog') if pred[0][0]>0 else plt.title('cat')
        plt.imshow(img[:,:,0],cmap='gray',vmin=0,vmax=1)
        plt.pause(0.5)
       
        
    

In [ ]:
plot_with_tag(sample_test_image[:10])

In [ ]:
x=history.history['accuracy']

In [ ]:
print(x)

In [ ]:
print(history)

In [ ]:
import pickle
with open('gray_tf_model/historyDict','wb') as file_pi:
    pickle.dump(history.history,file_pi)

In [ ]:
import pickle
history=pickle.load(open('gray_tf_model/historyDict','rb'))

In [ ]:
x=history['accuracy']
print(x)